In [ ]:
import os, openai
from dotenv import load_dotenv

load_dotenv()

# Set OpenAI configuration settings values
azure_openai_api_type        = os.environ["OPENAI_API_TYPE"]
azure_azure_openeai_key      = os.environ["OPENAI_API_KEY"]
azure_azur_openeai_endpoint  = os.environ["OPENAI_API_BASE"]
azure_openai_api_version     = os.environ["OPENAI_API_VERSION"]
azure_openai_api_model       = os.environ["OPENAI_API_MODEL"]

# Temperature & Tokens
azure_openai_api_temperature = 0
azure_openai_api_max_tokens  = 1000

In [ ]:
import pyodbc

def execute_sql_script(sql_script):
    # Create a connection string
    conn_str = (
        r'DRIVER={ODBC Driver 17 for SQL Server};'
        r"SERVER={az-vm-esi-labs};"
        r'DATABASE=master;'
        r'Trusted_Connection=yes;'
    )
    # Establish a connection with the database
    conn = pyodbc.connect(conn_str)

    # Create a cursor from the connection
    cursor = conn.cursor()

    # Execute the SQL script
    cursor.execute(sql_script)
    # Fetch the results
    results = cursor.fetchall()
    # Commit the transaction if no errors
    conn.commit()

    # Close the connection
    conn.close()
    # Return the results
    return results

In [ ]:
sql_script = "SELECT name FROM sys.databases"
sql_server = "az-vm-esi-labs"
execute_sql_script(sql_script)

In [ ]:
def GeneratePromptFilename(filedir,data,index):
    system_role = data['BicylOnPremPrompts'][0]['system_role'] 
    if index ==1:
        user_prompt =  data['BicylOnPremPrompts'][index]["createdatabase"]  
        filename = filedir + 'createdatabase.sql'    
    if index ==2:
        user_prompt =  data['BicylOnPremPrompts'][index]["createschemastg"]  
        filename = filedir + 'createschemastg.sql' 
    if index ==3:
        user_prompt =  data['BicylOnPremPrompts'][index]["createschemaprd"]  
        filename = filedir + 'createschemaprd.sql'  
    if index ==4:
        user_prompt =  data['BicylOnPremPrompts'][index]["createtemptable"]  
        filename = filedir + 'createtemptable.sql'  
    if index ==5:
        user_prompt =  data['BicylOnPremPrompts'][index]["createprdtable"]  
        filename = filedir + 'createprdtable.sql'   
    if index ==6:
        user_prompt =  data['BicylOnPremPrompts'][index]["loadstagingtable"]  
        filename = filedir + 'loadstagingtable.sql' 
    if index ==7:
        user_prompt =  data['BicylOnPremPrompts'][index]["loadprdtable"]  
        filename = filedir + 'loadprdtable.sql'
    if index ==8:
        user_prompt =  data['BicylOnPremPrompts'][index]["createprocedure"]  
        filename = filedir + 'createprocedure.sql'        
    if index ==9:
        user_prompt =  data['BicylOnPremPrompts'][index]["createview"]  
        filename = filedir + 'createview.sql'

    messages=[
        {
            "role": "system", "content": system_role
        },
        {
            "role": "user", "content": user_prompt
        }    
    ]
    return messages,filename

In [ ]:
def create_tsql_file(filename, content):
    with open(filename, 'w') as f:
        f.write(content)

In [ ]:
def SendRequestToAzureOpenAI(
         azure_openai_api_type
        ,azure_oai_endpoint
        ,azure_openai_api_version
        ,azure_oai_key
        ,azure_oai_model
        ,azure_openai_api_temperature
        ,azure_openai_api_max_tokens
        ,messages
        ,filename
    ):         
    try:    
        import pprint  
        # Set OpenAI configuration settings
        openai.api_type = azure_openai_api_type
        openai.api_base = azure_oai_endpoint
        openai.api_version = azure_openai_api_version
        openai.api_key = azure_oai_key
        # Send request to Azure OpenAI model
        print("Sending request for summary to Azure OpenAI endpoint...\n\n")
        response = openai.ChatCompletion.create(
            engine=azure_oai_model,
            temperature = azure_openai_api_temperature,
            max_tokens  = azure_openai_api_max_tokens,
            messages    = messages
        )
        output = []
        #output = response.choices[0].message.content + "\n"
        output = (response.choices[0].message.content).replace("```python","").replace("```","").strip()
        create_tsql_file(filename, output)
        #execute_sql_script(output)
        print(output)
    except Exception as ex:
        print(ex)

In [11]:
import json
# from pprint import pprint

json_file = 'bicycle_data_prompt.json'

with open(json_file) as json_data:
    data = json.load(json_data)

filedir = "F:\\Presentation\\Unlocking AI Potential with Prompt Engineering\\tsql\\"

index = 1
while index<= len(data['BicylOnPremPrompts']):
    messages,filename = GeneratePromptFilename(filedir,data,index)
    #print(messages)
    if __name__ == '__main__': 
        SendRequestToAzureOpenAI (
            azure_openai_api_type
            ,azure_azur_openeai_endpoint
            ,azure_openai_api_version
            ,azure_azure_openeai_key
            ,azure_openai_api_model
            ,azure_openai_api_temperature
            ,azure_openai_api_max_tokens
            ,messages
            ,filename
        )
    index+=1

Sending request for summary to Azure OpenAI endpoint...


IF NOT EXISTS (SELECT name FROM sys.databases WHERE name = 'azureopenai')
BEGIN
    CREATE DATABASE azureopenai;
END;
Sending request for summary to Azure OpenAI endpoint...


IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'stg')
BEGIN
    EXEC('CREATE SCHEMA stg')
END
GO
Sending request for summary to Azure OpenAI endpoint...


IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'prd')
BEGIN
    EXEC('CREATE SCHEMA prd')
END
GO
Sending request for summary to Azure OpenAI endpoint...


IF OBJECT_ID('stg.salestmp', 'U') IS NOT NULL
    DROP TABLE stg.salestmp;

CREATE TABLE stg.salestmp
(
    ProductId VARCHAR(255),
    ProductName VARCHAR(255),
    ProductType VARCHAR(255),
    Color VARCHAR(255),
    OrderQuantity VARCHAR(255),
    Size VARCHAR(255),
    Category VARCHAR(255),
    Country VARCHAR(255),
    Date VARCHAR(255),
    PurchasePrice VARCHAR(255),
    SellingPrice VARCHAR(255)
);
Sending request for summary to

UnboundLocalError: cannot access local variable 'user_prompt' where it is not associated with a value